In [9]:
import os
from pathlib import Path


In [ ]:
import os
import struct
import socket
import json
from pprint import pprint
import time
import threading


PORT = 64997
HOST = "localhost"


class SizeProtocol():
    MSGSIZE = 16

    def send(self, connected_socket, msg):
        connected_socket.send(struct.pack('I', len(msg)))
        connected_socket.sendall(msg.encode())

    def recv(self, connected_socket):
        msg_size, = struct.unpack('I', connected_socket.recv(struct.calcsize('I')))
        result = b''
        while len(result) < msg_size:
            part = connected_socket.recv(min(self.MSGSIZE, msg_size - len(result)))
            if not part:  # Если часть пустая, значит соединение закрыто
                raise ConnectionResetError("Connection closed by peer")
            result += part
        return result.decode()


class Server:
    def __init__(self, host=HOST, port=PORT):
        self.protocol = SizeProtocol()
        self.host = host
        self.port = port

    def get_dirs(self):
        result = {}
        paths = os.environ.get('PATH').split(os.pathsep)

        for dir in paths:
            if os.path.isdir(dir):
                files = os.listdir(dir)  # список всех файлов в директории
                exe_files = [f for f in files if os.access(os.path.join(dir, f), os.X_OK)]
                if len(exe_files) != 0:
                    result[dir] = exe_files
        return result

    def save_to_json(self, dictionary, filename):
        with open(filename, "w") as fh:
            json.dump(dictionary, fh)

    def handle_client(self, client_socket):
        try:
            request = self.protocol.recv(client_socket)
            print(f'Server received command {request}')

            # Обработка команды (например, можно добавить логику выполнения команды)
            response = 'Answer from SERVER: Command received'
            self.protocol.send(client_socket, response)
        except Exception as e:
            print(f"Error handling client: {e}")
        finally:
            client_socket.close()

    def start_server(self):
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_socket.bind((self.host, self.port))
        server_socket.listen(1)
        print("Server started and waiting for connections...")
        while True:
            client, addr = server_socket.accept()
            print(f"Connection from {addr}")
            threading.Thread(target=self.handle_client, args=(client,)).start()


class Client:
    def __init__(self, host=HOST, port=PORT):
        self.protocol = SizeProtocol()
        self.host = host
        self.port = port

    def send_command(self, command):
        try:
            client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            client_socket.connect((self.host, self.port))
            self.protocol.send(client_socket, command)
            answer = self.protocol.recv(client_socket)
            print(answer)
        except ConnectionResetError as e:
            print(f"Connection error: {e}")
        except Exception as e:
            print(f"Error sending command: {e}")
        finally:
            client_socket.close()


def main():
    server = Server()

    # Запускаем сервер в отдельном потоке
    server_thread = threading.Thread(target=server.start_server)
    server_thread.daemon = True
    server_thread.start()

    client = Client()

    command = input("Enter command (type 'EXIT' to quit): ")
    while command != 'EXIT':
        print(f'Your command: {command}')
        client.send_command(command)
        command = input("Enter command (type 'EXIT' to quit): ")

if __name__ == "__main__":
    main()